## Домашнее задание:
Разобраться с моделькой генерации текста, собрать самим или взять датасет с вебинара и обучить генератор.

In [ ]:
import tensorflow as tf

import numpy as np
import os
import time

In [ ]:
path_to_file = 'Гомер.Илиада.txt'

In [ ]:
!file *

sample_data:      directory
Гомер.Илиада.txt: ISO-8859 text, with CRLF line terminators


In [ ]:
text = open(path_to_file, 'rb').read().decode(encoding='ISO-8859-1')
# Delete Unicode spescial symbol \ufeff
text = text.replace('\ufeff', '')
text = text.encode('CP1252').decode('CP1251', errors='ignore')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 856925 characters


In [ ]:
print(text[:500])

ПЕСНЬ ПЕРВАЯ

МОР. ГНЕВ







Пой, богиня, про гнев Ахиллеса, Пелеева сына,

Гнев проклятый, страданий без счета принесший ахейцам,

Много сильных душ героев пославший к Аиду,

Их же самих на съеденье отдавший добычею жадным

5 Птицам окрестным и псам. Это делалось, волею Зевса,

С самых тех пор, как впервые, поссорясь, расстались враждебно

Сын Атрея, владыка мужей, и Пелид многосветлый.


Кто ж из бессмертных богов возбудил эту ссору меж ними?

Сын Лето и Зевса. Цар


In [ ]:
text = text + text

In [ ]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

101 unique characters


In [ ]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [ ]:
text_as_int, text, len(text_as_int), len(text)

(array([52, 42, 54, ...,  0,  1,  0]),
 'ПЕСНЬ ПЕРВАЯ\r\n\r\nМОР. ГНЕВ\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\nПой, богиня, про гнев Ахиллеса, Пелеева сына,\r\n\r\nГнев проклятый, страданий без счета принесший ахейцам,\r\n\r\nМного сильных душ героев пославший к Аиду,\r\n\r\nИх же самих на съеденье отдавший добычею жадным\r\n\r\n5 Птицам окрестным и псам. Это делалось, волею Зевса,\r\n\r\nС самых тех пор, как впервые, поссорясь, расстались враждебно\r\n\r\nСын Атрея, владыка мужей, и Пелид многосветлый.\r\n\r\n\r\nКто ж из бессмертных богов возбудил эту ссору меж ними?\r\n\r\nСын Лето и Зевса. Царем раздраженный, наслал он\r\n\r\n10 Злую болезнь на ахейскую рать. Погибали народы\r\n\r\nИз-за того, что Хриса-жреца Атрид обесчестил.\r\n\r\nТот к кораблям быстролетным ахейцев пришел, чтоб из плена\r\n\r\nВызволить дочь, за нее заплативши бесчисленный выкуп.\r\n\r\nШел, на жезле золотом повязку неся Аполлона,\r\n\r\n15 И обратился с горячей мольбою к собранью ахейцев,\r\n\r\nБольше всего же - к обо

In [ ]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [ ]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'ПЕСНЬ ПЕРВАЯ\r\n\r\nМОР. ГНЕВ\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\nПой, богиня, про гнев Ахиллеса, Пелеева сына,\r\n\r\nГнев прокля'
'тый, страданий без счета принесший ахейцам,\r\n\r\nМного сильных душ героев пославший к Аиду,\r\n\r\nИх же са'
'мих на съеденье отдавший добычею жадным\r\n\r\n5 Птицам окрестным и псам. Это делалось, волею Зевса,\r\n\r\nС'
' самых тех пор, как впервые, поссорясь, расстались враждебно\r\n\r\nСын Атрея, владыка мужей, и Пелид мно'
'госветлый.\r\n\r\n\r\nКто ж из бессмертных богов возбудил эту ссору меж ними?\r\n\r\nСын Лето и Зевса. Царем ра'


In [ ]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'ПЕСНЬ ПЕРВАЯ\r\n\r\nМОР. ГНЕВ\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\nПой, богиня, про гнев Ахиллеса, Пелеева сына,\r\n\r\nГнев прокл'
Target data: 'ЕСНЬ ПЕРВАЯ\r\n\r\nМОР. ГНЕВ\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\nПой, богиня, про гнев Ахиллеса, Пелеева сына,\r\n\r\nГнев прокля'


In [ ]:
# Batch size
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<_BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [ ]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           25856     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 lstm_1 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 lstm_2 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 lstm_3 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 dense (Dense)               (64, None, 101)           103525    
                                                                 
Total params: 30,554,469
Trainable params: 30,554,469
No

In [ ]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)


In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
!rm -rf ./training_checkpoints

In [ ]:
!ls ./training_checkpoints

ls: cannot access './training_checkpoints': No such file or directory


In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=7*10,
    save_weights_only=True)

In [ ]:
EPOCHS = 20
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
265/265 [==============================] - 111s 410ms/step - loss: 1.5373
Epoch 2/20
265/265 [==============================] - 116s 432ms/step - loss: 1.3648
Epoch 3/20
265/265 [==============================] - 109s 405ms/step - loss: 1.2558
Epoch 4/20
265/265 [==============================] - 111s 411ms/step - loss: 1.1622
Epoch 5/20
265/265 [==============================] - 110s 410ms/step - loss: 1.0678
Epoch 6/20
265/265 [==============================] - 111s 411ms/step - loss: 0.9696
Epoch 7/20
265/265 [==============================] - 110s 408ms/step - loss: 0.8669
Epoch 8/20
265/265 [==============================] - 116s 408ms/step - loss: 0.7660
Epoch 9/20
265/265 [==============================] - 111s 410ms/step - loss: 0.6719
Epoch 10/20
265/265 [==============================] - 110s 410ms/step - loss: 0.5901
Epoch 11/20
265/265 [==============================] - 111s 410ms/step - loss: 0.5252
Epoch 12/20
265/265 [==============================] - 109s 406

In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_20'

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 256)            25856     
                                                                 
 lstm_4 (LSTM)               (1, None, 1024)           5246976   
                                                                 
 lstm_5 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 lstm_6 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 lstm_7 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 dense_1 (Dense)             (1, None, 101)            103525    
                                                                 
Total params: 30,554,469
Trainable params: 30,554,469


In [ ]:
def generate_text(model, start_string, temperature=1, num_generate=500):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = num_generate

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = temperature

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [ ]:
text_ = generate_text(model, start_string=u"И с места поднялся Сын Атрея, герой Агамемнон пространнодержавный.", temperature=1)
print(text_)

И с места поднялся Сын Атрея, герой Агамемнон пространнодержавный.

После того как с орела кровавые снявши доспехи,

В руки немедля им плугом. И с ними вполне я о битве?

135 На поворот от грозный ид Талфам

Бывших в одной колеснице; побочный основа

Я говорю справедливо. Никак допустить ты не можешь,

Чтобы не достигнут он скорбел и за вожжи взялся, натянувши

Черную кровь вытирь искремертные боги,

135 Всех одинаково был он словах и щитами упало

Над троянцам бегом пробегал приказанья.

670 Больше еще лишь упорно, не зная усталости, битьс


In [ ]:
text_ = generate_text(model, start_string=u"И с места поднялся Сын Атрея, герой Агамемнон пространнодержавный.", temperature=0.5)
print(text_)

И с места поднялся Сын Атрея, герой Агамемнон пространнодержавный.

После того как питьем и едой утолили желанье,

Ткать размышления первым владыка мужей Агамемнон

И Одиссей многоумный. Барашков для клятвенной жертвы

Пару ягнят и вино, веселящее дух человека,

И не к богам принесли гекатомбы.

И, обращаясь к Пелиду, слова окрыленные молвил:

"Пулидамант! Удержи у себя тут храбрейших троянцев,

Я же отправлюсь туда и в общую битву вмешаюсь,

И, как слова они милой супруги исполнили скоро.

К первой со словом крылатым Зевес обратился к Приа
